In [10]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

# Project 1: Build Decision Tree(DV-"Survived",IDV-"Age,Gender and Fare") and Prediction

In [11]:
train = pd.read_csv('/home/pi/Lets_Upgrade/Dataset/train.csv')

In [12]:
train_age = np.where(train.Age.isnull(),train.Age.mean(), train.Age)

In [13]:
encoding = preprocessing.LabelEncoder()

In [14]:
train_gender = encoding.fit_transform(train.Sex)

In [15]:
features = pd.DataFrame([train_age, train.Fare, train_gender]).T

In [16]:
DT1 = tree.DecisionTreeClassifier(max_depth = 6)
DT1.fit(X = features, y = train.Survived)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [17]:
DT1.score(X = features, y = train.Survived)

0.8256467941507312

# Prediction

In [18]:
test = pd.read_csv('/home/pi/Lets_Upgrade/Dataset/test.csv')

In [19]:
test_age = np.where(test.Age.isnull(), test.Age.mean(), test.Age)

In [20]:
test_gender = encoding.fit_transform(test.Sex)

In [21]:
features_test = pd.DataFrame([test_age, test.Fare, test_gender]).T

In [22]:
prediction = DT1.predict(X = features_test)

In [23]:
pred_output = pd.DataFrame({"Passenger_Id": test.PassengerId, "Survived" : prediction})

In [24]:
pred_output.head()

,Passenger_Id,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [25]:
with open('DecisionTree1.dot','w') as f:
    file = tree.export_graphviz(DT1, feature_names=['AGE','FARE','GENDER'], out_file= f)

# Project 2: Build Decision Tree for Attrition Rate Analysis
## .DV - "Attrition"
## .IDV - Output of RF Algorithm

In [26]:
data = pd.read_csv('/home/pi/Lets_Upgrade/Dataset/general_data.csv')
data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4.0,Y,12,8,2,9.0,2,6,0,4


In [27]:
data['NumCompaniesWorked'] = np.where(data.NumCompaniesWorked.isnull(), data.NumCompaniesWorked.mean(), data.NumCompaniesWorked )
data['TotalWorkingYears'] = np.where(data.TotalWorkingYears.isnull(), data.TotalWorkingYears.mean(), data.TotalWorkingYears )

In [28]:
encoding = preprocessing.LabelEncoder()

In [29]:
data_temp = data.copy()

In [30]:
data_temp['Attrition'] = encoding.fit_transform(data.Attrition)
data_temp['BusinessTravel'] = encoding.fit_transform(data.BusinessTravel)
data_temp['Department'] = encoding.fit_transform(data.Department)
data_temp['EducationField'] = encoding.fit_transform(data.EducationField)
data_temp['Gender'] = encoding.fit_transform(data.Gender)
data_temp['JobRole'] = encoding.fit_transform(data.JobRole)
data_temp['MaritalStatus'] = encoding.fit_transform(data.MaritalStatus)
data_temp['Over18'] = encoding.fit_transform(data.Over18)

In [31]:
RF1 = RandomForestClassifier(n_estimators = 2000, max_features = 2, oob_score = True)

In [32]:
features = ['Age', 'BusinessTravel', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 
                'Gender', 'JobLevel', 'JobRole', 'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked', 
                'Over18', 'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 
                'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

In [33]:
RF1.fit(X = data_temp[features],y = data_temp.Attrition)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [34]:
RF1.oob_score_

1.0

In [35]:
for train_features, imp in zip(features,RF1.feature_importances_):
    print("train_features: {} ,'imp : {}'".format(train_features,imp))

train_features: Age ,'imp : 0.09682805889858338'
train_features: BusinessTravel ,'imp : 0.027910184914593454'
train_features: Department ,'imp : 0.026348088256277487'
train_features: DistanceFromHome ,'imp : 0.06953062445529502'
train_features: Education ,'imp : 0.04076774975487339'
train_features: EducationField ,'imp : 0.041468006830170115'
train_features: Gender ,'imp : 0.018412426464504403'
train_features: JobLevel ,'imp : 0.03780109603446629'
train_features: JobRole ,'imp : 0.05572599305364732'
train_features: MaritalStatus ,'imp : 0.03980058494616036'
train_features: MonthlyIncome ,'imp : 0.09329340315011156'
train_features: NumCompaniesWorked ,'imp : 0.05568512075458324'
train_features: Over18 ,'imp : 0.0'
train_features: PercentSalaryHike ,'imp : 0.06593272473708423'
train_features: StockOptionLevel ,'imp : 0.03428558737393445'
train_features: TotalWorkingYears ,'imp : 0.08497029400452824'
train_features: TrainingTimesLastYear ,'imp : 0.0452455855773391'
train_features: YearsAt

In [36]:
test_features = ['Age','DistanceFromHome', 'MonthlyIncome', 'PercentSalaryHike', 'TotalWorkingYears', 'YearsAtCompany']

In [37]:
DT2 = tree.DecisionTreeClassifier(max_depth=12)

In [38]:
DT2.fit(X = data_temp[test_features],y = data_temp.Attrition)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [39]:
DT2.score(X = data_temp[test_features], y = data_temp.Attrition)

0.9480725623582766

# 3. Build Decision Tree for Bank Loan Modelling
## .DV - "Personal Loan"
## .IDV - Output of RF Algorithm

In [40]:
data3 = pd.read_excel('/home/pi/Lets_Upgrade/Dataset/Bank_Personal_Loan_Modelling.xlsx', sheet_name='Data')
data3.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [41]:
RF2 = RandomForestClassifier(n_estimators=2000,max_features=2,oob_score=True)

In [42]:
train_feature = [ 'Age', 'Experience', 'Income', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard']

In [43]:
RF2.fit(X = data3[train_feature],y = data3['Personal Loan'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [44]:
RF2.oob_score_

0.9878

In [45]:
for train_features, imp in zip(train_feature,RF2.feature_importances_):
    print("train_features: {} ,'imp : {}'".format(train_features,imp))

train_features: Age ,'imp : 0.04464711423347098'
train_features: Experience ,'imp : 0.044218863072151594'
train_features: Income ,'imp : 0.3425524776260109'
train_features: Family ,'imp : 0.09882522635143572'
train_features: CCAvg ,'imp : 0.1850300674680777'
train_features: Education ,'imp : 0.16469686357919242'
train_features: Mortgage ,'imp : 0.0435830644013551'
train_features: Securities Account ,'imp : 0.005575098682353112'
train_features: CD Account ,'imp : 0.052785992467778244'
train_features: Online ,'imp : 0.008253099824523345'
train_features: CreditCard ,'imp : 0.009832132293651003'


In [46]:
test_features = ['Family', 'CCAvg', 'Education', 'CD Account','Age','Mortgage']

In [47]:
DT3 = tree.DecisionTreeClassifier(max_depth=12)

In [48]:
DT3.fit(X =data3[test_features],y = data3['Personal Loan'])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [49]:
DT3.score(X = data3[test_features], y = data3['Personal Loan'])

0.9866